In [1]:
import tensorflow as tf
import numpy as np
from scipy.misc import imread, imresize, imsave
import scipy.io
from scipy.io import savemat, loadmat
from os import listdir, walk
from os.path import isfile, join
#import PIL
import matplotlib.pyplot as plt
import time

start_time = time.time()

trainPath = 'data/train/'
avgImgHeight = int(384/4)       # Actual average height is 360. I am doing more downsampling
avgImgWidth = int(384/4)        # Actual average width is 400. I am doing more downsampling
imgClasses = ['dog', 'cat']
numClasses = len(imgClasses)

fileNames = next(walk(trainPath))[2]
numTrain = len(fileNames)
trainDataX = np.zeros((numTrain, avgImgHeight, avgImgWidth, 3), dtype=np.uint8)
trainDataY = np.zeros((numTrain, numClasses), dtype=np.uint8)
for i, fileName in enumerate(fileNames):
    img = imread(join(trainPath, fileName))
    img = imresize(img, (avgImgHeight, avgImgWidth))
    trainDataX[i, :, :, :] = img

    for j, imgClass in enumerate(imgClasses):
        trainDataY[i, j] = (fileName[0:3] == imgClass)

print('Time taken to load data = ', time.time()-start_time)

Time taken to load data =  122.58116340637207


In [2]:
learningRate = 1e-5
numIter = 10000
batchSize = 100

X = tf.placeholder(tf.float32, [None, avgImgHeight, avgImgWidth, 3])
y = tf.placeholder(tf.float32, [None, numClasses])


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.01)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


W_conv1 = weight_variable([5,5,3,32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(X, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([int(avgImgHeight/4 * avgImgWidth/4 * 64), 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, int(avgImgHeight/4 * avgImgWidth/4 * 64)])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, numClasses])
b_fc2 = bias_variable([numClasses])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(learningRate).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

sess = tf.Session()

sess.run(tf.initialize_all_variables())

for i in range(numIter):
    sampleIndices = np.random.choice(np.arange(numTrain), batchSize)
    batch_xs = trainDataX[sampleIndices]   #
    batch_ys = trainDataY[sampleIndices]
    sess.run(train_step, feed_dict={X: batch_xs, y: batch_ys, keep_prob: 0.5})
    if i%10 == 0:
        print("Iteration = ", i, "Cross Entropy = ", sess.run(cross_entropy, feed_dict={X:batch_xs, y: batch_ys, keep_prob: 1.0}),
              "Train Accuracy = ", sess.run(accuracy, feed_dict={X:batch_xs, y: batch_ys, keep_prob: 1.0}))

print("Final Train Accuracy = ", sess.run(accuracy, feed_dict={X: trainDataX, y: trainDataY, keep_prob: 1.0}))


Iteration =  0 Cross Entropy =  0.683961 Train Accuracy =  0.55
Iteration =  10 Cross Entropy =  0.732297 Train Accuracy =  0.52
Iteration =  20 Cross Entropy =  0.658776 Train Accuracy =  0.62
Iteration =  30 Cross Entropy =  0.635595 Train Accuracy =  0.76
Iteration =  40 Cross Entropy =  0.64381 Train Accuracy =  0.67
Iteration =  50 Cross Entropy =  0.696507 Train Accuracy =  0.55
Iteration =  60 Cross Entropy =  0.65166 Train Accuracy =  0.63


KeyboardInterrupt: 